In [1]:
import os
QPATH = "Quantlet/5-domain-pre-training"
os.environ["WANDB_DISABLED"] = "true"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
import sys
IN_COLAB = 'google.colab' in sys.modules

import os

if IN_COLAB:
  os.chdir(f'/content/drive/MyDrive/ColabNotebooks/IRTG/Encode_the_Qode/Encode-the-Qode/{QPATH}')
  #%pip install protobuf==3.20.1
  %pip install transformers[torch]
  %pip install -q sentencepiece
  %pip install datasets==2.13.1
  %pip install evaluate
  %pip install rouge_score
  #%pip install git+https://github.com/huggingface/nlp.git@fix-bad-type-in-overflow-check

sys.path.append('../4-qode2desc')

In [3]:
import pickle
import json
import re
import sys
from IPython.display import display

from tqdm import tqdm
import pandas as pd
import numpy as np
tqdm.pandas()


import matplotlib.pyplot as plt
import seaborn as sns

import torch
from torch.utils.data import  DataLoader
from transformers import AutoTokenizer, AutoModelWithLMHead, SummarizationPipeline
from transformers import AdamW
from datasets import load_dataset

from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq
)

import nltk
nltk.download('punkt')
import evaluate

import importlib
import datetime

from sklearn.model_selection import train_test_split

import analysis_modules
importlib.reload(analysis_modules)

[nltk_data] Downloading package punkt to
[nltk_data]     /home/RDC/zinovyee.hub/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/RDC/zinovyee.hub/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/RDC/zinovyee.hub/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


<module 'analysis_modules' from '/usr/net/zinovyee.hub/IRTG/MLSC/Encode-the-Qode/Quantlet/5-domain-pre-training/../4-qode2desc/analysis_modules.py'>

In [6]:
model_name = "CodeT5"
SAMPLE = 'test'


# tokenization
encoder_max_length = 300
decoder_max_length = 75

RS = 42
LR = 5e-5

EPOCHS = 2
TRAIN_BATCH = 4
EVAL_BATCH  = 4

WARMUP_STEPS  = 100
WEIGHT_DECAY  = 0.1
LOGGING_STEPS = 100
SAVE_TOTAL_LIM = 1

LABEL_SMOOTHING  = 0.1
PREDICT_GENERATE = True

EVAL_COLUMNS = ['eval_loss',
                'eval_rouge1',
                'eval_rouge2',
                'eval_rougeL',
                'eval_rougeLsum',
                'eval_bleu',
                'eval_gen_len']

DATE = str(datetime.date.today())
analysis_name = model_name + '-'  + SAMPLE  + '-' + str(EPOCHS) + '-' + str(encoder_max_length) + '-' + str(TRAIN_BATCH)

analysis_name = analysis_name + '-' + DATE

print(analysis_name)

CodeT5-test-2-300-4-2023-11-03


In [7]:
print(analysis_name)
analysis_modules.scs_analyze(analysis_name=analysis_name,
                                    model_name=model_name,
                                    train_data_path=f'../../data/preprocessed/ArXiv/',
                                    train_data_name='arxiv_pretrain_train.json',
                                    val_data_path=f'../../data/preprocessed/ArXiv/',
                                    val_data_name='arxiv_pretrain_test.json',
                                    encoder_max_length=encoder_max_length,
                                    decoder_max_length=decoder_max_length,
                                    random_state=RS,
                                    eval_columns_list=EVAL_COLUMNS,
                                    learning_rate=LR,
                                    epochs=EPOCHS,
                                    train_batch=TRAIN_BATCH,
                                    eval_batch=EVAL_BATCH,
                                    warmup_steps=WARMUP_STEPS,
                                    weight_decay=WEIGHT_DECAY,
                                    logging_stes=LOGGING_STEPS,
                                    save_total_lim=SAVE_TOTAL_LIM,
                                    label_smooting=LABEL_SMOOTHING,
                                    predict_generate=PREDICT_GENERATE,
                                    save_strategy='epoch',
                                    load_best_model_at_end=False,
                                    freeze=True)

CodeT5-test-2-300-4-2023-11-03
CodeT5-test-2-300-4-2023-11-03
cuda
cuda


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/39326 [00:00<?, ? examples/s]

Map:   0%|          | 0/16855 [00:00<?, ? examples/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


   eval_loss  eval_rouge1  eval_rouge2  eval_rougeL  eval_rougeLsum  \
0      5.419        0.173        0.054         0.15            0.15   

   eval_bleu  eval_gen_len  
0      0.004        18.456  


/home/RDC/zinovyee.hub/.local/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
9832,3.769400
19664,3.465200


   eval_loss  eval_rouge1  eval_rouge2  eval_rougeL  eval_rougeLsum  \
0      3.284        0.415        0.228        0.374           0.374   

   eval_bleu  eval_gen_len  
0      0.103        14.873  
__________
Original: Comparing Different Deep Learning Architectures for Classification of Chest Radiographs


Summary before Tuning: This function is used to classify chest radiographs.


Summary after Tuning: Deep Scale Chest Radiographs: A Comparative Study on Deep Learning


__________



__________
Original: TGHop: An Explainable, Efficient and Lightweight Method for Texture Generation


Summary before Tuning: This method is a lightweight method that is used to generate a high quality texture for a sequence of images of a specific type.


Summary after Tuning: TGHop: A Survey


__________



__________
Original: Unidentified Floating Object detection in maritime environment using dictionary learning


Summary before Tuning: Maritime domain is one of the most challenging scenarios for